In [5]:
pip install mlxtend

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.4 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 5.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

# Завантаження датасету
df = pd.read_csv('clinicHeart.csv')

# Перевірка на пропущені значення
df.fillna(0, inplace=True)

# Перетворення числових даних у бінарний формат (0/1)
df_bin = df.copy()
for col in df_bin.columns:
    if df_bin[col].nunique() > 2:  # Якщо більше двох унікальних значень
        df_bin[col] = pd.cut(df_bin[col], bins=2, labels=[0, 1], duplicates='drop')

df_bin = df_bin.astype(bool).astype(int)  # Перетворюємо у формат 0/1

# Перевіряємо, чи всі значення тепер 0 або 1
assert df_bin.isin([0, 1]).all().all(), "Не всі значення 0 або 1!"

### Пункт 1: Бібліотечна реалізація Apriori
frequent_itemsets_apriori = apriori(df_bin, min_support=0.05, use_colnames=True)
rules_apriori = association_rules(frequent_itemsets_apriori, metric="confidence", min_threshold=0.5)
print("\n=== Пункт 1: Бібліотечна реалізація Apriori ===")
print("Apriori Rules:\n", rules_apriori)

### Пункт 2: Бібліотечна реалізація FP-Growth
frequent_itemsets_fpgrowth = fpgrowth(df_bin, min_support=0.05, use_colnames=True)
rules_fpgrowth = association_rules(frequent_itemsets_fpgrowth, metric="confidence", min_threshold=0.5)
print("\n=== Пункт 2: Бібліотечна реалізація FP-Growth ===")
print("FP-Growth Rules:\n", rules_fpgrowth)

### Пункт 3: Власна реалізація Apriori
def apriori_custom(dataset, min_support=0.05):
    """ Власна реалізація алгоритму Apriori """
    def support_count(itemset):
        return sum(dataset[list(itemset)].all(axis=1))

    # Генерація 1-елементних наборів
    items = {frozenset([col]): support_count([col]) for col in dataset.columns}
    items = {k: v / len(dataset) for k, v in items.items() if v / len(dataset) >= min_support}
    
    frequent_itemsets = list(items.items())  # Часті набори

    # Генерація наборів більшої довжини
    def generate_candidates(itemsets, k):
        candidates = set()
        for i in range(len(itemsets)):
            for j in range(i + 1, len(itemsets)):
                union_set = itemsets[i][0] | itemsets[j][0]
                if len(union_set) == k:
                    candidates.add(union_set)
        return candidates

    # Рекурсивна генерація частих наборів
    k = 2
    while True:
        candidates = generate_candidates(frequent_itemsets, k)
        new_itemsets = {itemset: support_count(list(itemset)) / len(dataset) for itemset in candidates}
        new_itemsets = {k: v for k, v in new_itemsets.items() if v >= min_support}
        
        if not new_itemsets:
            break
        
        frequent_itemsets.extend(new_itemsets.items())
        k += 1

    return frequent_itemsets

# Виклик власного Apriori
frequent_itemsets_apriori_custom = apriori_custom(df_bin, min_support=0.05)
print("\n=== Пункт 3: Apriori (Власна реалізація) ===")
print("Apriori Custom Frequent Itemsets:\n", frequent_itemsets_apriori_custom)

### Пункт 4: Власна реалізація FP-Growth
class FPNode:
    def __init__(self, item, count, parent=None):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}

def fpgrowth_custom(dataset, min_support=0.05):
    """ Власна реалізація алгоритму FP-Growth """
    def build_fptree(dataset, min_support):
        # Підрахунок частоти елементів
        item_counts = dataset.sum(axis=0)
        item_counts = item_counts[item_counts / len(dataset) >= min_support]
        item_counts = item_counts.sort_values(ascending=False)

        # Фільтрація стовпців, які не відповідають мінімальній підтримці
        dataset = dataset[item_counts.index]

        # Створення кореневого вузла дерева
        root = FPNode(None, 1)

        # Створення FP-дерева
        for _, transaction in dataset.iterrows():
            transaction = set(transaction[transaction == 1].index)
            current_node = root
            for item in sorted(transaction, key=lambda x: item_counts[x], reverse=True):
                if item not in current_node.children:
                    current_node.children[item] = FPNode(item, 1, current_node)
                else:
                    current_node.children[item].count += 1
                current_node = current_node.children[item]
        
        return root, item_counts

    def mine_fp_tree(node, prefix, item_counts, min_support, frequent_itemsets):
        # Збирання частих множин з FP-дерева
        if node.item:
            frequent_itemsets.append((prefix + [node.item], node.count))
        
        # Рекурсивний обхід дітей
        for item, child in node.children.items():
            mine_fp_tree(child, prefix + [item], item_counts, min_support, frequent_itemsets)
    
    # Побудова дерева та видобуток частих множин
    root, item_counts = build_fptree(dataset, min_support)
    frequent_itemsets = []
    mine_fp_tree(root, [], item_counts, min_support, frequent_itemsets)

    return frequent_itemsets

# Виклик власного FP-Growth
frequent_itemsets_fpgrowth_custom = fpgrowth_custom(df_bin, min_support=0.05)
print("\n=== Пункт 4: FP-Growth (Власна реалізація) ===")
print("FP-Growth Custom Frequent Itemsets:\n", frequent_itemsets_fpgrowth_custom)

### Пункт 5: Власна реалізація Eclat
def eclat(dataset, min_support=0.05):
    """ Реалізація алгоритму Eclat """
    def support_count(itemset):
        return sum(dataset[list(itemset)].all(axis=1))

    items = {frozenset([col]): support_count([col]) for col in dataset.columns}
    items = {k: v / len(dataset) for k, v in items.items() if v / len(dataset) >= min_support}
    
    if not items:  # Якщо немає частих множин — повертаємо порожній список
        return []

    def recursive_eclat(prefix, items):
        """ Рекурсивна генерація частих множин """
        while items:
            item, support = items.popitem()
            yield (prefix | item, support)
            new_items = {
                other_item: support_count(prefix | item | other_item) / len(dataset)
                for other_item in items.keys()
            }
            new_items = {k: v for k, v in new_items.items() if v >= min_support}
            if new_items:
                yield from recursive_eclat(prefix | item, new_items)
    
    return list(recursive_eclat(set(), items))

# Виклик Eclat
eclat_rules = eclat(df_bin, min_support=0.05)
print("\n=== Пункт 5: Eclat (Власна реалізація) ===")
print("Eclat Rules:\n", eclat_rules)

C:\Users\Asus\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\Asus\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(



=== Пункт 1: Бібліотечна реалізація Apriori ===
Apriori Rules:
                                 antecedents           consequents  \
0                                     (age)        (serum_sodium)   
1                                     (age)                 (sex)   
2                       (ejection_fraction)             (anaemia)   
3                     (high_blood_pressure)             (anaemia)   
4                                 (anaemia)        (serum_sodium)   
..                                      ...                   ...   
206           (anaemia, time, smoking, sex)        (serum_sodium)   
207  (anaemia, time, smoking, serum_sodium)                 (sex)   
208   (anaemia, sex, smoking, serum_sodium)                (time)   
209                (anaemia, time, smoking)   (sex, serum_sodium)   
210                 (anaemia, sex, smoking)  (time, serum_sodium)   

     antecedent support  consequent support  support  confidence      lift  \
0                0.2714     